In [ ]:
!pip install -q tfds-nightly

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/ModelCNN_WEBIA.keras')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
builder = tfds.builder("eurosat")

In [ ]:
tf.random.set_seed(42)

ds, info = tfds.load("eurosat",
                    split="train",
                    with_info=True,
                    builder_kwargs={"config": "rgb"})  # We only use Sentinl 2 RGB data
                                                       # These are bands 2, 3, and 4

In [ ]:
def preprocess(sample):
    image = sample['image']
    label = sample['label']


    # Here, we normalize the data to make it easier to process
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [ ]:
batch_size = 32
total_examples = info.splits["train"].num_examples
train_size = int(0.7 * total_examples)
val_size = int(0.2 * total_examples)
test_size = total_examples - train_size - val_size


train_ds = (ds.take(train_size)
            .map(preprocess)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE))

val_ds = (ds.skip(train_size)
          .take(val_size)
          .map(preprocess)
          .batch(batch_size)
          .prefetch(tf.data.AUTOTUNE))

test_ds = (ds.skip(train_size + val_size)
           .map(preprocess)
           .batch(batch_size)
           .prefetch(tf.data.AUTOTUNE))


AUTOTUNE=tf.data.AUTOTUNE; BATCH=64
def cfg(ds,train=False):
    if train: ds=ds.shuffle(10_000,seed=42)
    return ds.batch(BATCH).prefetch(AUTOTUNE)
ds_train=cfg(train_ds,True); ds_val=cfg(val_ds); ds_test=cfg(test_ds)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(64, 64, 3)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"), # We use Convolution layers to detect pattern
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.GlobalAveragePooling2D(),

    tf.keras.layers.Dense(256, activation='relu'), # Dense layer (each perceptron is connected to another from the Flatten layer), ReLU activation keeps only positive values
    tf.keras.layers.Dropout(0.3), # Sets 30% of weights to 0 to prevent overfitting
    tf.keras.layers.Dense(10, activation='softmax') # Dense layer that outputs class probabilities for the 10 categories (the sum of the probabilities equals 1; that's what softmax does)
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

callbacks=[
  tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=8,restore_best_weights=True),
  tf.keras.callbacks.ModelCheckpoint("best.keras",monitor="val_accuracy",save_best_only=True),
]


history = model.fit(train_ds, validation_data=val_ds, epochs=40)

test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Créer une figure avec deux sous-graphiques côte à côte
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Graphe de la perte
axes[0].plot(history.history['loss'], label='Train Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_title('Loss')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].legend()

# Graphe de l'accuracy
axes[1].plot(history.history['accuracy'], label='Train Accuracy')
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[1].set_title('Accuracy')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Accuracy')
axes[1].legend()

# Afficher les graphiques
plt.tight_layout()
plt.show()

model.save('/content/drive/MyDrive/ModelCNN_WEBIA.keras')

model.evaluate(test_ds, verbose=0)
